In [11]:
"""
5) Calcular los intervalos de confianza para cada uno de los coeficientes de regresión con un intervalo de
confianza del 75% y explica el resultado obtenido.

"""



'\nLa empresa productora de vinos cree que es posible construir un modelo de mejor calidad que pueda predecir\nmejor el grado de alcohol de los futuros vinos si se utilizan varios atributos del dataset. Para ello se solicita la\ncreación de nuevos modelos realizando las siguientes acciones:\n\n1) Crear los diferentes conjuntos de entrenamiento y test para el dataset.\n\n2) Construir dos modelos de regresión que utilicen al menos 4 de los atributos (es posible utilizar más\natributos) disponibles en el dataset mediante la utilización de un proceso aprendizaje iterativo.\n\n3) Calcular el error de cada uno de los modelos y realizar una comparación entre ambos para identificar\nel modelo que mejor resultado ofrece. Es necesario justificar la respuesta mediante la\nutilización de datos empíricos.\n\n4) Evaluar el modelo seleccionado con el conjunto de test construido anteriormente y explicar el\nresultado obtenido.\n\n5) Calcular los intervalos de confianza para cada uno de los coeficiente

# Paso 1: Creamos los diferentes conjuntos de entrenamiento y test

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm


data = pd.read_csv('wine_alcohol.csv', delimiter=';')

train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

# Paso 2: Construimos dos modelos de regresión utilizando al menos 4 de los atributos
## Modelo 1 con todas las variables

In [13]:
features1 = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "pH", "sulphates", "wine type", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "quality"]
X_train1 = train_set[features1]
y_train = train_set["alcohol"]
X_test1 = test_set[features1]
y_test = test_set["alcohol"]


model1 = LinearRegression()
model1.fit(X_train1, y_train)
predictions1 = model1.predict(X_train1)
mse1 = mean_squared_error(y_train, predictions1)
print(f'MSE Modelo 1: {mse1}')

Model 1 MSE: 0.25873672693490407


## Modelo 2 con las variables más significativas

In [14]:
features2 = ["chlorides", "density", "sulphates", "wine type", "fixed acidity", "residual sugar"]
X_train2 = train_set[features2]
X_test2 = test_set[features2]

model2 = LinearRegression()
model2.fit(X_train2, y_train)
predictions2 = model2.predict(X_train2)
mse2 = mean_squared_error(y_train, predictions2)
print(f'MSE Modelo 2: {mse2}')

Model 2 MSE: 0.3771582593602398


# Paso 3: Comparamos el error y elegimos el mejor de entre los dos

In [15]:
if mse1 < mse2:
    print("El Modelo 1 ofrece mejor resultado")
    best_model = model1
    best_X_train = X_train1
    best_X_test = X_test1
else:
    print("El Modelo 2 ofrece mejor resultado")
    best_model = model2
    best_X_train = X_train2
    best_X_test = X_test2

Model 1 performs better.


Obtenemos que el modelo 1 ofrece el mejor resultado, esto lo sabemos porque presenta un MSE (diferencia cuadrada promedio entre los valores predichos por el modelo y los valores reales observados en el conjunto de datos) más bajo (0.25873672693490407 < 0.3771582593602398), lo que indica que el modelo tiene un mejor ajuste a los datos, ya que las predicciones están más cerca de los valores reales. Esto es de esperar ya que el modelo 1 lo hemos construido empleando todas las variables independientes, mientras que el modelo 2 unicamente con las variables más significativas. 
Si tuvieramos una base de datos con muchas observaciones, a pesar de que el modelo 1 ofrece un mejor resultado, computacionalmente podría compensar utilizar el modelo 2 ya que la diferencia del MSE no es tan significativa, y el ahorro en tiempo de computación al reducir las variables estudiadas a la mitad sí podría ser considerable.

# Paso 4: Evaluamos el modelo elegido en el conjunto de test

In [16]:
final_predictions = best_model.predict(best_X_test)
final_mse = mean_squared_error(y_test, final_predictions)
print(f'Modelo Final MSE: {final_mse}')

Final Model MSE: 0.20360683331490026


Un MSE de 0.2036 sugiere que las predicciones del modelo se desvían, en promedio, por aproximadamente 0.2036 unidades de alcohol respecto a los valores reales, es decir, presenta una generalización efectiva respecto a nuevos datos.
El MSE ha mejorado en el conjunto de test en comparación con el conjunto de entrenamiento, indicando que el modelo es capaz de hacer predicciones precisas en situaciones del mundo real y puede ser confiable para su implementación en aplicaciones prácticas.

# Paso 5: Cálculo de intervalos de los coeficientes con una confianza del 75%

In [20]:
# Seleccionar el mejor modelo
best_features = features1

# Añadir una columna de unos para el término de intersección
X_train_best = sm.add_constant(X_train1)

# Ajustar el modelo usando statsmodels para obtener los resultados
model_sm = sm.OLS(y_train, X_train_best)
results = model_sm.fit()
print(results.summary())


# Calcular y mostrar los intervalos de confianza para los coeficientes
# El nivel de confianza del 75% implica un alpha de 0.25
print("Intervalos de confianza al 75% para cada coeficiente:")
intervals = results.conf_int(alpha=0.25)
print(intervals)

                            OLS Regression Results                            
Dep. Variable:                alcohol   R-squared:                       0.818
Model:                            OLS   Adj. R-squared:                  0.817
Method:                 Least Squares   F-statistic:                     1937.
Date:                Mon, 06 May 2024   Prob (F-statistic):               0.00
Time:                        20:47:07   Log-Likelihood:                -3861.2
No. Observations:                5197   AIC:                             7748.
Df Residuals:                    5184   BIC:                             7834.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  641.4624 

Observamos en el resumen OLS un R cuadrado de 0.818, e indica que el modelo explica el 81.8% de la variabilidad en la variable dependiente. Es una medida de qué tan bien las predicciones se aproximan a los datos reales, siendo bastante considerable.
Por último respecto a los intervalos de confianza de los coeficientes, un intervalo de confianza al 75% significa que, si se repitieran los muestreos y el análisis muchas veces, el coeficiente verdadero estaría dentro del intervalo especificado aproximadamente el 75% de las veces.
El intervalo de confianza de los coeficientes nos proporciona una estimación del rango en el que podemos esperar encontrar el valor verdadero del coeficiente en la población, basado en la muestra de datos que tenemos. Por ejemplo, el coeficiente de fixed acidity varía entre 0.499314 y 0.516599, esto sugiere que, manteniendo todo lo demás constante, un aumento de una unidad en la fixed acidity está asociado con un aumento en el grado de alcohol entre estos dos valores. Los valores que tienen un intervalo cercano a 0 sugieren que no son significativas respecto de la variable dependiente, alcohol. Por lo general cuadran bastante los intervalos, ya que como hemos dicho en el apartado 3, las variables que generan un MSE menor, tienen un intervalo de confianza alto, sin embargo esto no es completamente exacto, ya que en el caso del pH esto no se cumple, y eso es porque no se tiene en cuenta la correlación no lineal, es decir, al aparecer junto a otras variables específicas, la correlación de determinadas variables cambia.